In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import matplotlib.pyplot as plt
from data_grouping import *
import torch
import numpy as np

While staggering the start point may be a good idea, I won't implement it inside the dataset object...

Also only going to create the stacked dataset (as encode/decode is not particularly relevant for me)



https://pytorch.org/docs/stable/data.html#loading-batched-and-non-batched-data

https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

he has a more sustainable solution that doesn't require you to have the full dataset in memory.  The way I would use this would be to make a folder called `data` and it would contain a csv for each sample series and then a metadata csv to hold onto the different parameters used to make each series (they could also be npy files)

In [3]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
    '''Characterizes a dataset for PyTorch'''
    def __init__(self, data):
        'Initialization'
        self.data = data
        
    def __len__(self):
        'Denotes the total number of samples'
        return self.data.x.shape[0]

    def __getitem__(self, index):
        'Generates one sample of data, these can then be concatenated by dataloader instance'

        # Load data and get label
        X = self.data.x[index]
        y = self.data.y[index]

        return X,y

train split https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets/50544887#50544887

https://discuss.pytorch.org/t/how-to-split-dataset-into-test-and-validation-sets/33987/2


transforms https://stackoverflow.com/questions/50002543/what-are-transforms-in-pytorch-used-for

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [33]:
from load_sines import *

sin_data = data_grouping(sines, input_len=2,output_len=4,tau_offset=3,inputs_overlap=False)

sines.shape, freqs.shape, sin_data.x.shape

TypeError: __init__() takes from 2 to 6 positional arguments but 7 were given

In [26]:
n_samples = sines.shape[0]-15.4
print(n_samples)
n_train = int(n_samples * 0.8)
n_test  = int(n_samples * 0.1)
n_valid = n_samples - n_train - n_test
n_train + n_valid + n_test

99984.6


99984.6

In [92]:
training_set = Dataset(sin_data)

params = {'batch_size': 20102,
          'shuffle': True,
          'drop_last': False,
          'num_workers': 11}

training_generator = data.DataLoader(training_set, **params)


In [93]:
for X,y in training_generator:
    print(X.shape, y.shape)

torch.Size([20102, 497, 2]) torch.Size([20102, 497, 4])
torch.Size([20102, 497, 2]) torch.Size([20102, 497, 4])
torch.Size([20102, 497, 2]) torch.Size([20102, 497, 4])
torch.Size([20102, 497, 2]) torch.Size([20102, 497, 4])
torch.Size([19592, 497, 2]) torch.Size([19592, 497, 4])


Call this segmented or partitioned or isolated sequences... note that we no longer have to worry about overlapping.. in fact we want them to because it increases the amount of data the model sees (note that 1,2 would never have been seen before if input was 2 and we were stacking).. also we don't have to worry about staggering.  called segmented because the parent seq is segmented into many little ones

also note that the first dimension is partially redundant if you don't care about keeping track of which parent seq each fragment comes from... if you want to get rid of it you can just say .reshape(-1,input_len) and similarly for the output. Why you'd want to keep it? Scoring. Why not? Training, you'd have to program with that extra dimension that means nothing (I suggest getting rid of it to train using the above and then putting it back with .reshape(batch_size,-1,output_len) at the end)

In [1]:
def running_view(arr, window, axis=-1):
    """
    return a running view of length 'window' over 'axis'
    the returned array has an extra last dimension, which spans the window
    """
    shape = list(arr.shape)
    shape[axis] -= (window-1)
    assert(shape[axis]>0)
    return np.lib.index_tricks.as_strided(
        arr,
        shape + [window],
        arr.strides + (arr.strides[axis],))

In [22]:
x_t = np.arange(40).reshape(-1,10)
x_t

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]])

In [24]:
from segmented_data import *

In [35]:
segmented_univariant_data(x_t,input_len=3,output_len=2).x.reshape(-1,3).reshape(4,-1,3)

tensor([[[ 0.,  1.,  2.],
         [ 1.,  2.,  3.],
         [ 2.,  3.,  4.],
         [ 3.,  4.,  5.],
         [ 4.,  5.,  6.],
         [ 5.,  6.,  7.]],

        [[10., 11., 12.],
         [11., 12., 13.],
         [12., 13., 14.],
         [13., 14., 15.],
         [14., 15., 16.],
         [15., 16., 17.]],

        [[20., 21., 22.],
         [21., 22., 23.],
         [22., 23., 24.],
         [23., 24., 25.],
         [24., 25., 26.],
         [25., 26., 27.]],

        [[30., 31., 32.],
         [31., 32., 33.],
         [32., 33., 34.],
         [33., 34., 35.],
         [34., 35., 36.],
         [35., 36., 37.]]])

In [23]:
running_view(x_t,3)

array([[[ 0,  1,  2],
        [ 1,  2,  3],
        [ 2,  3,  4],
        [ 3,  4,  5],
        [ 4,  5,  6],
        [ 5,  6,  7],
        [ 6,  7,  8],
        [ 7,  8,  9]],

       [[10, 11, 12],
        [11, 12, 13],
        [12, 13, 14],
        [13, 14, 15],
        [14, 15, 16],
        [15, 16, 17],
        [16, 17, 18],
        [17, 18, 19]],

       [[20, 21, 22],
        [21, 22, 23],
        [22, 23, 24],
        [23, 24, 25],
        [24, 25, 26],
        [25, 26, 27],
        [26, 27, 28],
        [27, 28, 29]],

       [[30, 31, 32],
        [31, 32, 33],
        [32, 33, 34],
        [33, 34, 35],
        [34, 35, 36],
        [35, 36, 37],
        [36, 37, 38],
        [37, 38, 39]]])